d
# Chapter 9: Building Reliable Data Lakes with Delta Lake and Apache Spark™

Delta Lake: An open-source storage format that brings ACID transactions to Apache Spark™ and big data workloads.

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>


* **Open format**: Stored as Parquet format in blob storage.
* **ACID Transactions**: Ensures data integrity and read consistency with complex, concurrent data pipelines.
* **Schema Enforcement and Evolution**: Ensures data cleanliness by blocking writes with unexpected.
* **Audit History**: History of all the operations that happened in the table.
* **Time Travel**: Query previous versions of the table by time or version number.
* **Deletes and upserts**: Supports deleting and upserting into tables with programmatic APIs.
* **Scalable Metadata management**: Able to handle millions of files are scaling the metadata operations with Spark.
* **Unified Batch and Streaming Source and Sink**: A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box. 

### Source:
This notebook is a modified version of the [SAIS EU 2019 Delta Lake Tutorial](https://github.com/delta-io/delta/tree/master/examples/tutorials/saiseu19). The data used is a modified version of the public data from [Lending Club](https://www.kaggle.com/wendykan/lending-club-loan-data). It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Loading data in Delta Lake table

First let’s, read this data and save it as a Delta Lake table.

In [0]:
spark.sql("set spark.sql.shuffle.partitions = 1")

sourcePath = "/databricks-datasets/learning-spark-v2/loans/loan-risks.snappy.parquet"

# Configure Delta Lake Path
deltaPath = "/tmp/loans_delta"

# Remove folder if it exists
dbutils.fs.rm(deltaPath, recurse=True)

# Create the Delta table with the same loans data
(spark.read.format("parquet").load(sourcePath) 
  .write.format("delta").save(deltaPath))

spark.read.format("delta").load(deltaPath).createOrReplaceTempView("loans_delta")
print("Defined view 'loans_delta'")

Let's explore the data.

In [0]:
spark.sql("SELECT count(*) FROM loans_delta").show()

In [0]:
spark.sql("SELECT * FROM loans_delta LIMIT 5").show()

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Loading data streams into Delta Lake table

We will generate a stream of data from with randomly generated loan ids and amounts. 
In addition, we are going to define a few useful utility functions.

In [0]:
import random
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *

def random_checkpoint_dir(): 
  return "/tmp/chkpt/%s" % str(random.randint(0, 10000))


# User-defined function to generate random state

states = ["CA", "TX", "NY", "WA"]

@udf(returnType=StringType())
def random_state():
  return str(random.choice(states))


# Function to start a streaming query with a stream of randomly generated data and append to the parquet table
def generate_and_append_data_stream():

  newLoanStreamDF = (spark.readStream.format("rate").option("rowsPerSecond", 5).load() 
    .withColumn("loan_id", 10000 + col("value")) 
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer")) 
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000)) 
    .withColumn("addr_state", random_state())
    .select("loan_id", "funded_amnt", "paid_amnt", "addr_state"))
    
  checkpointDir = random_checkpoint_dir()

  streamingQuery = (newLoanStreamDF.writeStream 
    .format("delta") 
    .option("checkpointLocation", random_checkpoint_dir()) 
    .trigger(processingTime = "10 seconds") 
    .start(deltaPath))

  return streamingQuery

# Function to stop all streaming queries 
def stop_all_streams():
  # Stop all the streams
  print("Stopping all streams")
  for s in spark.streams.active:
    s.stop()
  print("Stopped all streams")
  print("Deleting checkpoints")  
  dbutils.fs.rm("/tmp/chkpt/", True)
  print("Deleted checkpoints")

In [0]:
streamingQuery = generate_and_append_data_stream()

You can see that the streaming query is adding data to the table by counting the number of records in the table. Run the following cell multiple times.

In [0]:
spark.sql("SELECT count(*) FROM loans_delta").show()

**Remember to stop all the streaming queries.**

In [0]:
stop_all_streams()

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Enforcing schema on write to prevent data corruption

 Let’s test this by trying to write some data with an additional column `closed` that signifies whether the loan has been terminated. Note that this column does not exist in the table.

In [0]:
cols = ['loan_id', 'funded_amnt', 'paid_amnt', 'addr_state', 'closed']

items = [
  (1111111, 1000, 1000.0, 'TX', True), 
  (2222222, 2000, 0.0, 'CA', False)
]

from pyspark.sql.functions import *

loanUpdates = (spark
                .createDataFrame(items, cols)
                .withColumn("funded_amnt", col("funded_amnt").cast("int")))

In [0]:
# Uncomment the line below and it will error
# loanUpdates.write.format("delta").mode("append").save(deltaPath)

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Evolving schema to accommodate changing data

In [0]:
(loanUpdates.write.format("delta").mode("append")
  .option("mergeSchema", "true")
  .save(deltaPath))

Let's query the table once again to see the schema.

In [0]:
spark.read.format("delta").load(deltaPath).filter("loan_id = 1111111").show()

For existing rows are read, the value of the new column is considered as NULL.

In [0]:
spark.read.format("delta").load(deltaPath).show()

##  ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Transforming existing data


Let's look into how we can transform existing data. But first, let's refine the view on the table because the schema has changed and the view needs to redefined to pick up the new schema.

In [0]:
spark.read.format("delta").load(deltaPath).createOrReplaceTempView("loans_delta")
print("Defined view 'loans_delta'")

#### Updating loan data to fix errors
* Upon reviewing the data, we realized that all of the loans assigned to `addr_state = 'OR'` should have been assigned to `addr_state = 'WA'`.
* In Parquet, to do an `update`, you would need to 
  * Copy all of the rows that are not affected into a new table
  * Copy all of the rows that are affected into a DataFrame, perform the data modification
  * Insert the previously noted DataFrame's rows into the new table
  * Remove the old table
  * Rename the new table to the old table

In [0]:
spark.sql("""SELECT addr_state, count(1) FROM loans_delta WHERE addr_state IN ('OR', 'WA', 'CA', 'TX', 'NY') GROUP BY addr_state""").show()

Let's fix the data.

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, deltaPath)
deltaTable.update("addr_state = 'OR'",  {"addr_state": "'WA'"})

Let's see the data once again.

In [0]:
spark.sql("""SELECT addr_state, count(1) FROM loans_delta WHERE addr_state IN ('OR', 'WA', 'CA', 'TX', 'NY') GROUP BY addr_state""").show()

#### Deleting user-related data from a table General Data Protection Regulation (GDPR)

You can remove data that matches a predicate from a Delta Lake table. Let's say we want to remove all the fully paid loans. Let's first see how many are there.

In [0]:
spark.sql("SELECT COUNT(*) FROM loans_delta WHERE funded_amnt = paid_amnt").show()

Now let's delete them.

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, deltaPath)
deltaTable.delete("funded_amnt = paid_amnt")

Let's check the number of fully paid loans once again.

In [0]:
spark.sql("SELECT COUNT(*) FROM loans_delta WHERE funded_amnt = paid_amnt").show()

### Upserting change data to a table using merge
A common use cases is Change Data Capture (CDC), where you have to replicate row changes made in an OLTP table to another table for OLAP workloads. To continue with our loan data example, say we have another table of new loan information, some of which are new loans and others are updates to existing loans. In addition, let’s say this changes table has the same schema as the loan_delta table. You can upsert these changes into the table using the DeltaTable.merge() operation which is based on the MERGE SQL command.

In [0]:
spark.sql("select * from loans_delta where addr_state = 'NY' and loan_id < 30").show()

Let's say we have some changes to this data, one loan has been paid off, and another new loan has been added.

In [0]:
cols = ['loan_id', 'funded_amnt', 'paid_amnt', 'addr_state', 'closed']

items = [
  (11, 1000, 1000.0, 'NY', True),   # loan paid off
  (12, 1000, 0.0, 'NY', False)      # new loan
]

loanUpdates = spark.createDataFrame(items, cols)

Now, let's update the table with the change data using the `merge` operation.

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, deltaPath)

(deltaTable
  .alias("t")
  .merge(loanUpdates.alias("s"), "t.loan_id = s.loan_id") 
  .whenMatchedUpdateAll() 
  .whenNotMatchedInsertAll() 
  .execute())

Let's see whether the table has been updated.

In [0]:
spark.sql("select * from loans_delta where addr_state = 'NY' and loan_id < 30").show()

### Deduplicating data while inserting using insert-only merge

The merge operation in Delta Lake supports an extended syntax beyond that specified by the ANSI standard. It supports advanced features like the following. 
- Delete actions: For example, `MERGE … WHEN MATCHED THEN DELETE`
- Clause conditions: For example, `MERGE … WHEN MATCHED AND <condition> THEN ...``
- Optional actions: All the MATCHED and NOT MATCHED clauses are optional.
- Star syntax: For example, `UPDATE *` and `INSERT *` to update/insert all the columns in the target table with matching columns from the source dataset. The equivalent API in DeltaTable is `updateAll()` and `insertAll()`, which we have already seen.

This allows you to express many more complex use cases with little code. For example, say you want to backfill the loan_delta table with historical data of past loans. But some of the historical data may already have been inserted in the table and you don't want to update them (since their emails may already have been updated). You can deduplicate by the loan_id while inserting by running the following merge operation with only the INSERT action (since the UPDATE action is optional).

In [0]:
spark.sql("select * from loans_delta where addr_state = 'NY' and loan_id < 30").show()

Let's say we have some historical data that we want to merge with this table. One of the historical loan exists in the current table but the historical table has old values, therefore it should not update the current value present in the table. And another historical does not exist in the current table, therefore it should be inserted into the table.

In [0]:
cols = ['loan_id', 'funded_amnt', 'paid_amnt', 'addr_state', 'closed']

items = [
  (11, 1000, 0.0, 'NY', False),
  (-100, 1000, 10.0, 'NY', False)
]

historicalUpdates = spark.createDataFrame(items, cols)

Let's do the merge.

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, deltaPath)

(deltaTable
  .alias("t")
  .merge(historicalUpdates.alias("s"), "t.loan_id = s.loan_id") 
  .whenNotMatchedInsertAll() 
  .execute())

Let's see whether the table has been updated.

In [0]:
spark.sql("select * from loans_delta where addr_state = 'NY' and loan_id < 30").show()

Notice that the only change in the table is that insert of new loan, and existing loans were not updated to old values.

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Auditing data changes with operation history

All changes to the Delta table are recorded as commits in the table's transaction log. As you write into a Delta table or directory, every operation is automatically versioned. You can use the HISTORY command to view the table's history.

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, deltaPath)
deltaTable.history().show()

In [0]:
deltaTable.history(4).select("version", "timestamp", "operation", "operationParameters").show(truncate=False)

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Querying previous snapshots of the table with time travel

Delta Lake’s time travel feature allows you to access previous versions of the table. Here are some possible uses of this feature:

* Auditing Data Changes
* Reproducing experiments & reports
* Rollbacks

You can query by using either a timestamp or a version number using Python, Scala, and/or SQL syntax. For this examples we will query a specific version using the Python syntax.  

For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html) and the [docs](https://docs.delta.io/latest/delta-batch.html#deltatimetravel).

**Let's query the table's state before we deleted the data, which still contains the fully paid loans.**

In [0]:
previousVersion = deltaTable.history(1).select("version").first()[0] - 3

(spark.read.format("delta")
  .option("versionAsOf", previousVersion)
  .load(deltaPath)
  .createOrReplaceTempView("loans_delta_pre_delete"))

spark.sql("SELECT COUNT(*) FROM loans_delta_pre_delete WHERE funded_amnt = paid_amnt").show()

We see the same number of fully paid loans that we had seen before delete.